# **Project Objective and Brief**

## *In this project, rule-based and Deep-Learning algorithms are used with an aim to first appropriately detect different type of emotions contained in a collection of Tweets and then accurately predict the overall emotions of the Tweets is done.*

## **Preprocessor is a preprocessing library used for tweet data written in Python.While building Machine Learning systems based on tweet data, a preprocessing is required. This library makes it easy to clean, parse or tokenize the tweets.The same is imported here.** 

In [ ]:
!pip install tweet-preprocessor 2>/dev/null 1>/dev/null

## **Importing Libraries**

In [ ]:
import preprocessor as pcr
import numpy as np 
import pandas as pd 
import emoji
import keras
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from sklearn import preprocessing,  model_selection
from keras.preprocessing import sequence, text
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
import plotly.express as px
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tokenizers import Tokenizer, models 
from tensorflow.keras.layers import SpatialDropout1D

# **Data preparation**

In [ ]:
df_data_1 = pd.read_csv("../input/tweetscsv/Tweets.csv")

In [ ]:
df_data_1.head()

In [ ]:
df_data = df_data_1[["tweet_id","airline_sentiment","text"]]

In [ ]:
df_data.head()

#  **Correcting Spelling of data**

In [ ]:
data_spell = pd.read_csv("../input/spelling/aspell.txt",sep=":",names=["correction","misspell"])
data_spell.misspell = data_spell.misspell.str.strip()
data_spell.misspell = data_spell.misspell.str.split(" ")
data_spell = data_spell.explode("misspell").reset_index(drop=True)
data_spell.drop_duplicates("misspell",inplace=True)
miss_corr = dict(zip(data_spell.misspell, data_spell.correction))

#Sample of the dict
{v:miss_corr[v] for v in [list(miss_corr.keys())[k] for k in range(20)]}

In [ ]:
def correct_spell(v):
    for a in v.split(): 
        if a in miss_corr.keys(): 
            v = v.replace(a, miss_corr[a]) 
    return v

df_data["clean_content"] = df_data.text.apply(lambda a : correct_spell(a))

# **Using a Python library for expanding and creating common English contractions in text**

In [ ]:
contract = pd.read_csv("../input/contractions/contractions.csv")
cont_dict = dict(zip(contract.Contraction, contract.Meaning))

In [ ]:
def contract_to_meaning(v): 
  
    for a in v.split(): 
        if a in cont_dict.keys(): 
            v = v.replace(a, cont_dict[a]) 
    return v


In [ ]:
df_data.clean_content = df_data.clean_content.apply(lambda a : contract_to_meaning(a))

# **Removal of URLs and Mentions from dataset**

In [ ]:
pcr.set_options(pcr.OPT.MENTION, pcr.OPT.URL)
pcr.clean("hello guys @alx #sport🔥 1245 https://github.com/s/preprocessor")

In [ ]:
df_data["clean_content"]=df_data.text.apply(lambda a : pcr.clean(a))

# **Removal of Punctuations and Emojis from dataset**

In [ ]:
def punct(v): 
  
    punct = '''()-[]{};:'"\,<>./@#$%^&_~'''
  
    for a in v.lower(): 
        if a in punct: 
            v = v.replace(a, " ") 
    return v


In [ ]:
punct("test @ #ldfldlf??? !! ")

In [ ]:
df_data.clean_content = df_data.clean_content.apply(lambda a : ' '.join(punct(emoji.demojize(a)).split()))

In [ ]:
def text_cleaning(v):
    v = correct_spell(v)
    v = contract_to_meaning(v)
    v = pcr.clean(v)
    v = ' '.join(punct(emoji.demojize(v)).split())
    
    return v

In [ ]:
text_cleaning("isn't 💡 adultry @ttt good bad ... ! ? ")

# **Removing empty comments from dataset**

In [ ]:
df_data = df_data[df_data.clean_content != ""]

In [ ]:
df_data.airline_sentiment.value_counts()

# **Data Modeling**

## **Encoding the data and train, test and split it**

In [ ]:
id_for_sentiment = {"neutral":0, "negative":1,"positive":2}

In [ ]:
df_data["sentiment_id"] = df_data['airline_sentiment'].map(id_for_sentiment)

In [ ]:
df_data.head()

In [ ]:
encoding_label = LabelEncoder()
encoding_integer = encoding_label.fit_transform(df_data.sentiment_id)

encoding_onehot = OneHotEncoder(sparse=False)
encoding_integer = encoding_integer.reshape(len(encoding_integer), 1)
Y = encoding_onehot.fit_transform(encoding_integer)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_data.clean_content,Y, random_state=1995, test_size=0.2, shuffle=True)

# **LSTM: Long short-term memory** 

### **It is an artificial recurrent neural network (RNN) architecture used in the field of deep learning.**

In [ ]:
# using keras tokenizer here
tkn = text.Tokenizer(num_words=None)
maximum_length = 160
Epoch = 15
tkn.fit_on_texts(list(X_train) + list(X_test))
X_train_pad = sequence.pad_sequences(tkn.texts_to_sequences(X_train), maxlen=maximum_length)
X_test_pad = sequence.pad_sequences(tkn.texts_to_sequences(X_test), maxlen=maximum_length)

In [ ]:
t_idx = tkn.word_index

In [ ]:
embedding_dimension = 160
lstm_out = 250

model_sql = Sequential()
model_sql.add(Embedding(len(t_idx) +1 , embedding_dimension,input_length = X_test_pad.shape[1]))
model_sql.add(SpatialDropout1D(0.2))
model_sql.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model_sql.add(keras.layers.core.Dense(3, activation='softmax'))
#adam rmsprop 
model_sql.compile(loss = "categorical_crossentropy", optimizer='adam',metrics = ['accuracy'])
print(model_sql.summary())

In [ ]:
size_of_batch = 32

# **LSTM Model**

In [ ]:
model_sql.fit(X_train_pad, y_train, epochs = Epoch, batch_size=size_of_batch,validation_data=(X_test_pad, y_test))

In [ ]:
def get_emotion(model_sql,text_1):
    text_1 = text_cleaning(text_1)
    #tokenize
    tweet = tkn.texts_to_sequences([text_1])
    tweet = sequence.pad_sequences(tweet, maxlen=maximum_length, dtype='int32')
    emotion = model_sql.predict(tweet,batch_size=1,verbose = 2)
    emo = np.round(np.dot(emotion,100).tolist(),0)[0]
    rslt = pd.DataFrame([id_for_sentiment.keys(),emo]).T
    rslt.columns = ["sentiment","percentage"]
    rslt=rslt[rslt.percentage !=0]
    return rslt

In [ ]:
def result_plotting(df):
    #colors=['#D50000','#000000','#008EF8','#F5B27B','#EDECEC','#D84A09','#019BBD','#FFD000','#7800A0','#098F45','#807C7C','#85DDE9','#F55E10']
    #fig = go.Figure(data=[go.Pie(labels=df.sentiment,values=df.percentage, hole=.3,textinfo='percent',hoverinfo='percent+label',marker=dict(colors=colors, line=dict(color='#000000', width=2)))])
    #fig.show()
    clrs={'neutral':'rgb(213,0,0)','negative':'rgb(0,0,0)',
                    'positive':'rgb(0,142,248)'}
    col={}
    for i in rslt.sentiment.to_list():
        col[i]=clrs[i]
    figure = px.pie(df, values='percentage', names='sentiment',color='sentiment',color_discrete_map=col,hole=0.3)
    figure.show()

# **Result of LSTM**

### Paragraph-1

In [ ]:
rslt =get_emotion(model_sql,"Had an absolutely brilliant day ðŸ˜ loved seeing an old friend and reminiscing")
result_plotting(rslt)

# **Result of LSTM**

### Paragraph-2

In [ ]:
rslt =get_emotion(model_sql,"The pain my heart feels is just too much for it to bear. Nothing eases this pain. I can’t hold myself back. I really miss you")
result_plotting(rslt)

# **Result of LSTM**

### Paragraph-3

In [ ]:
rslt =get_emotion(model_sql,"I hate this game so much,It make me angry all the time ")
result_plotting(rslt)

# **LSTM with GloVe 6B 200d word embedding**
### **GloVe algorithm is an extension to the word2vec method for efficiently learning word vectors**

In [ ]:
def data_reading(file):
    with open(file,'r') as z:
        word_vocabulary = set() 
        word_vector = {}
        for line in z:
            line_1 = line.strip() 
            words_Vector = line_1.split()
            word_vocabulary.add(words_Vector[0])
            word_vector[words_Vector[0]] = np.array(words_Vector[1:],dtype=float)
    print("Total Words in DataSet:",len(word_vocabulary))
    return word_vocabulary,word_vector

In [ ]:
vocabulary, word_to_index =data_reading("../input/glove-global-vectors-for-word-representation/glove.6B.200d.txt")

In [ ]:
matrix_embedding = np.zeros((len(t_idx) + 1, 200))
for word, i in t_idx.items():
    vector_embedding = word_to_index.get(word)
    if vector_embedding is not None:
        matrix_embedding[i] = vector_embedding

In [ ]:
embedding_dimension = 200
lstm_out = 250

model_lstm = Sequential()
model_lstm.add(Embedding(len(t_idx) +1 , embedding_dimension,input_length = X_test_pad.shape[1],weights=[matrix_embedding],trainable=False))
model_lstm.add(SpatialDropout1D(0.2))
model_lstm.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model_lstm.add(keras.layers.core.Dense(3, activation='softmax'))
#adam rmsprop 
model_lstm.compile(loss = "categorical_crossentropy", optimizer='adam',metrics = ['accuracy'])
print(model_lstm.summary())

In [ ]:
size_of_batch = 32

# **LSTM with GloVe Model**

In [ ]:
model_lstm.fit(X_train_pad, y_train, epochs = Epoch, batch_size=size_of_batch,validation_data=(X_test_pad, y_test))

# **Result of LSTM GloVe**

### Paragraph-1

In [ ]:
rslt =get_emotion(model_lstm,"Had an absolutely brilliant day ðŸ˜ loved seeing an old friend and reminiscing")
result_plotting(rslt)

# **Result of LSTM GloVe**

### Paragraph-2

In [ ]:
rslt =get_emotion(model_lstm,"The pain my heart feels is just too much for it to bear. Nothing eases this pain. I can’t hold myself back. I really miss you")
result_plotting(rslt)

# **Result of LSTM GloVe**

### Paragraph-3

In [ ]:
rslt =get_emotion(model_lstm,"I hate this game so much,It make me angry all the time ")
result_plotting(rslt)

# **Conclusion**



**Algorithms used to detect different types of emotion from paragraph are**

**1- LSTM (Long Short Term Memory)**-It is an artificial recurrent neural network (RNN) architecture used in the field of deep learning.
**2- LSTM GloVe- GloVe algorithm is an extension to the word2vec method for efficiently learning word vectors.**

It has been concluded that using LSTM algorithm it is easier to classify the Tweets and a more accurate result is obtained.